In [115]:
import numpy as np
from Bio import SeqIO
from Bio.PDB import PDBParser, protein_letters_3to1
import requests
import os
import torch

#### Load CASP target list

This is a file with casp Regular Targets (T) together with a corresponding pdb code (if available)

In [89]:
with open('../../data/casp13_test_data/casp13_targets') as f:
    raw = f.readlines()
    
casp_targets = {}
for i in raw:
    line = i.strip().split()
    casp_targets[line[0]] = line[1]
    
del raw

`T0970` and `T1011` are fragmented

In [90]:
del casp_targets['T0951']
del casp_targets['T0965']
del casp_targets['T0970']
del casp_targets['T1011']

#### Load CASP target sequences

downloaded from: http://predictioncenter.org/download_area/CASP13/sequences/casp13.seq.txt

In [91]:
casp_sequences = {}
for record in SeqIO.parse("../../data/casp13_test_data/casp13_seq.fa", "fasta"):
    if record.id in list(casp_targets.keys()):
        casp_sequences[record.id] = str(record.seq)

## 1. Download PDB files

In [5]:
def download_pdb(target):
    url = 'https://files.rcsb.org/download/' + target + '.pdb'
    myfile = requests.get(url)
    open(f'../../data/casp13_test_data/pdbfiles/' + target + '.pdb', 'wb').write(myfile.content)

In [6]:
# DONE
#for t in casp_targets.values():
#    download_pdb(t)

## 2. Generate Sequences and Distance maps

In [92]:
def align(smaller, larger):
    '''Align smaller Sequence to larger sequence and return the start and end indices'''
    
    start = -1
    max_score = 0
    for i in range(len(larger) - len(smaller)+1):
        temp_score = 0
        for j in range(len(smaller)):
            if smaller[j] == larger[i+j]:
                temp_score += 1
                    
        if temp_score > max_score:
            max_score = temp_score
            start = i
                
    return start

In [93]:
def get_coords(target, cid=0, raw=False):
    
    L = len(casp_sequences[target])
    protein = casp_targets[target]
    
    # Get PDB structure
    try:
        structure = PDBParser().get_structure('', f'../../data/casp13_test_data/pdbfiles/{protein}.pdb')
    except (IndexError, ValueError):
        print('IndexError or ValueError')
        return
    
    # Get the first chain

    chain = structure[0].child_list[cid]

    coords_list = []

    known_aminoacids = np.array(list(protein_letters_3to1.keys()))

    for i, residue in enumerate(chain.get_residues()):
        residue_name = residue.get_resname()

        if residue_name not in known_aminoacids:
            break

        residue_oneletter = protein_letters_3to1[residue_name]
        residue_number = residue.child_list[0].get_full_id()[3][1]

        if residue_oneletter == 'G':  # if Glycin -> C-alpha. Otherwise C-beta
            try:
                atom = residue['CA']
            except KeyError:
                print('Missing C-alpha atom')
                return
        else:
            try:
                atom = residue['CB']
            except KeyError:
                print('Missing C-beta atom')
                return

        x, y, z = atom.get_coord()
        coords_list.append([residue_number,
                            residue_name,
                            residue_oneletter,
                            x, y, z])

    coords_list = np.array(coords_list, dtype='O')
    coords_start, coords_end = coords_list[0][0], coords_list[-1][0]
    
    if raw: 
        return coords_list
    
    # Fill missing data in the PDB coordinates
    missing = np.setdiff1d(np.arange(coords_list[:, 0][0], 1 + coords_list[:, 0][-1]), coords_list[:, 0]).astype(np.int)
    if len(missing) > 0:
        missingarr = np.zeros((len(missing), 6), dtype='O')
        missingarr[:, 0] = missing
        missingarr[:, 2] = np.repeat('X', len(missing))

        coords_list = np.concatenate((coords_list, missingarr))
        coords_list = coords_list[coords_list[:, 0].argsort()]
    
    pdbseq = ''.join(coords_list[:, 2])
    caspseq = casp_sequences[target]
    
    # Align the PDB file with the casp Sequence
    if pdbseq != caspseq:
        
        fill_before = min(L, coords_start)
        pdbseqX = 'X' * fill_before + pdbseq + 'X' * L
    
        if fill_before > 0:
            coords_list = np.concatenate((
                np.array([[coords_start - fill_before + i, 0, 'X', 0, 0, 0] for i in range(fill_before)], dtype='O'),
                coords_list,
                np.array([[coords_end + i + 1, 0, 'X', 0, 0, 0] for i in range(L)], dtype='O')
            ))
        else:
            coords_list = np.concatenate((
                coords_list,
                np.array([[coords_end + i + 1, 0, 'X', 0, 0, 0] for i in range(L)], dtype='O')
            ))
                
        start = align(caspseq, pdbseqX)
        
        coords_list = coords_list[start:(start + L)]
    
        for i in range(len(coords_list)):
            if coords_list[i, 2] != 'X':
                if caspseq[i] != coords_list[i, 2]:
                    coords_list[i] = [coords_list[i, 0], 0, 'X', 0, 0, 0]

    return coords_list

In [104]:
def dist_mat(target):
    """ Generates distance matrix with 64 and 32 bins as well as the sequence"""

    coords = get_coords(target)

    L = coords.shape[0]

    dist_mat = np.zeros((L, L))

    for i in range(L):
        for j in range(L):
            if coords[i, 2] == 'X' or coords[j, 2] == 'X':
                dist_mat[i, j] = -1
            else:
                dist_mat[i, j] = np.sqrt(np.sum((coords[i, 3:] - coords[j, 3:]) ** 2))

    bins64 = np.concatenate(([0], np.linspace(2, 22, 62), [1000]))
    bins32 = np.concatenate(([0], np.linspace(2, 22, 30), [1000]))

    dist_mat64 = np.digitize(dist_mat, bins64)
    dist_mat32 = np.digitize(dist_mat, bins32)

    return dist_mat64, dist_mat32

In [113]:
def make_fasta(target):
    sequence = casp_sequences[target]
    with open(f'../../data/casp13_test_data/sequences/{target}.fasta', 'w') as s:
        s.write(f'>{target}\n{sequence}')

In [119]:
#for t in casp_targets:
#    d64, d32 = dist_mat(t)
#    make_fasta(t)
#    torch.save(torch.from_numpy(d64), f'../../data/casp13_test_data/distance_maps/distance_maps64/{t}.pt')
#    torch.save(torch.from_numpy(d32), f'../../data/casp13_test_data/distance_maps/distance_maps32/{t}.pt')

## DONE!!!

**SANITY CHECKS**

In [96]:
for t in casp_targets:
    pdbl = len(''.join(coords[t][:, 2]))
    caspl = len(casp_sequences[t])
    print(f'Target {t}, PDB length: {pdbl}, CASP length: {caspl}')

Target T1018, PDB length: 334, CASP length: 334
Target T1016, PDB length: 203, CASP length: 203
Target T1014, PDB length: 276, CASP length: 276
Target T1009, PDB length: 718, CASP length: 718
Target T1008, PDB length: 80, CASP length: 80
Target T1006, PDB length: 79, CASP length: 79
Target T1005, PDB length: 364, CASP length: 364
Target T1003, PDB length: 474, CASP length: 474
Target T1000, PDB length: 523, CASP length: 523
Target T0990, PDB length: 552, CASP length: 552
Target T0984, PDB length: 752, CASP length: 752
Target T0976, PDB length: 252, CASP length: 252
Target T0971, PDB length: 186, CASP length: 186
Target T0969, PDB length: 487, CASP length: 487
Target T0967, PDB length: 81, CASP length: 81
Target T0966, PDB length: 494, CASP length: 494
Target T0963, PDB length: 372, CASP length: 372
Target T0961, PDB length: 505, CASP length: 505
Target T0960, PDB length: 384, CASP length: 384
Target T0958, PDB length: 96, CASP length: 96
Target T0955, PDB length: 41, CASP length: 41
Ta

In [97]:
for t in casp_targets:
    print(f'Target: {t}\nCasp Sequence:\n{casp_sequences[t]}\nPDB Sequence:\n{"".join(coords[t][:, 2])}\n\n')    

Target: T1018
Casp Sequence:
MITSSLPLTDLHRHLDGNIRTQTILELGQKFGVKLPANTLQTLTPYVQIVEAEPSLVAFLSKLDWGVAVLGDLDACRRVAYENVEDALNARIDYAELRFSPYYMAMKHSLPVTGVVEAVVDGVRAGVRDFGIQANLIGIMSRTFGTDACQQELDAILSQKNHIVAVDLAGDELGQPGDRFIQHFKQVRDAGLHVTVHAGEAAGPESMWQAIRDLGATRIGHGVKAIHDPKLMDYLAQHRIGIESCLTSNLQTSTVDSLATHPLKRFLEHGILACINTDDPAVEGIELPYEYEVAAPQAGLSQEQIRQAQLNGLELAFLSDSEKKALLAKAALRG
PDB Sequence:
MITSSLPLTDLHRHLDGNIRTQTILELGQKFGVKLPANTLQTLTPYVQIVEAEPSLVAFLSKLDWGVAVLGDLDACRRVAYENVEDALNARIDYAELRFSPYYMAMKHSLPVTGVVEAVVDGVRAGVRDFGIQANLIGIMSRTFGTDACQQELDAILSQKNHIVAVDLAGDELGQPGDRFIQHFKQVRDAGLHVTVHAGEAAGPESMWQAIRDLGATRIGHGVKAIHDPKLMDYLAQHRIGIESCLTSNLQTSTVDSLATHPLKRFLEHGILACINTDDPAVEGIELPYEYEVAAPQAGLSQEQIRQAQLNGLELAFLSDSEKKALLAKAALRG


Target: T1016
Casp Sequence:
MRLWLIRHGETQANIDGLYSGHAPTPLTARGIEQAQNLHTLLHGVSFDLVLCSELERAQHTARLVLSDRQLPVQIIPELNEMFFGDWEMRHHRDLMQEDAENYSAWCNDWQHAIPTNGEGFQAFSQRVERFIARLSEFQHYQNILVVSHQGVLSLLIARLIGMPAEAMWHFRVDQGCWSAIDINQKFATLRVLNSRAIGVENA
PDB Sequence:
MRLWLIRHGETQANIDGLYSGHAPTPLTARGIEQAQNL